In [ ]:
import numpy as np

`vv1` and `vv2` are to be given so that `vv1[i]` and `vv2[i]` were images of orthogonal VPs in the i-th picture. Coefficients are calculated in the notebook "Compute linear system coefficients for omega".

Recall that $\omega = (KK^T)^{-1} = K^{-T}K^{-1}$. Since the inverse of an upper triangular matrix is upper triangular, $K^{-T}$ is lower triangular, so the Cholesky decomposition of $\omega$ will return $K^{-T}$, which will have to be inverted and transposed.

In [ ]:
def K_from_vps(vv1, vv2):
    assert (len(vv1) == 4 and len(vv2) == 4), "Length of vv1 and vv2 must be 4!"
    A = []
    b = []
    for i in range(len(vv1)):
        A += [[ vv1[i][0]*vv2[i][0],
                vv1[i][0]*vv2[i][2] + vv1[i][2]*vv2[i][0],
                vv1[i][1]*vv2[i][2] + vv1[i][2]*vv2[i][1],
                vv1[i][1]*vv2[i][1] ]]
        b += [ -vv1[i][2]*vv2[i][2] ]
    w = np.linalg.solve(A, b)
    
    W = np.array([[w[0],  0,   w[1]],
                 [ 0,  w[3],  w[2]],
                 [w[1], w[2],  1 ]])
    K = np.linalg.inv(np.linalg.cholesky(np.array(W))).T
    return K/K[2,2]

**TODO:** Test this with ground truth data

In [ ]:
with open("pics2/vps.csv") as f:
    vps = np.array([l.strip("\n").split(",") for l in f.readlines()]).astype("float")
vv1, vv2 = vps[:,:3], vps[:,3:]
vv1[:, 0:2] = vv1[:, 0:2]/6000
vv2[:, 0:2] = vv2[:, 0:2]/6000
K_from_vps(vv1, vv2)

Consider a coordinate system centered in the center of the table, with $x$ and $y$ directions parallel to its edges. Once we have $K$, $R$ can be computed by considering that the rays through the projections of the vanishing points must be perpendicular.

Let $\mathbf u$ and $\mathbf v$ be the projections vanishing points on the plane $z=k$ in Euclidean coordinates (these can be obtained by inverting $K$ on the pixel coordinates). We know $\mathbf u \cdot \mathbf v = 0$, so solving for $k$ we get $k^2 = -x_u x_v - y_u y_v$. Immediately we get $\mathbf x' = \frac{\mathbf u}{||\mathbf u||}$ and $\mathbf y' = \frac {\mathbf v} {||\mathbf v||}$, and of course $\mathbf z' = \mathbf x' \times \mathbf y'$. Lastly,

$$
R = \begin{pmatrix} 
\\
\mathbf x' & \mathbf y' & \mathbf z' \\
\\
\end{pmatrix}
$$.

In order to find the missing $\mathbf t$ that completes $P = K \begin{pmatrix}
R & \mathbf t
\end{pmatrix}$ the table size and the projections of its corners on the image plane can be used. There are five points of which we know in-world position and projections on the image plane: the four corners of the table and its center, the intersection of the diagonals. Let $\mathbf x'$ be the projection on the image plane of the known point $\mathbf x$, and let $H := \begin{pmatrix}
R & \mathbf t
\end{pmatrix}$. Being $\mathbb R^3$ the codomain of $H$ we can express the constraint that $H \mathbf x$ be in the span of $\mathbf x'$ with the equation $H \mathbf x \times \mathbf x' = \mathbf 0$. By writing $\mathbf x =: \begin{pmatrix}\mathbf x_{1-3} & x_4\end{pmatrix}^\top$ and considering the definition of $H$ one can rewrite the constraints as:

$$
(R \mathbf x_{1-3} + \mathbf t x_4) \times \mathbf x' = \mathbf 0 \\
\implies R \mathbf x_{1-3} \times \mathbf x' = -x_4 \mathbf t \\
\implies \mathbf x' \times \mathbf t = -\frac{1}{x_4} \mathbf x' \times R \mathbf x_{1-3}
$$

In the last form of the equation $\mathbf t$ is the only unknown, as the right hand side is a constant vector; furthermore, cross product by $\mathbf x'$ can be expressed as a matrix multiplication, hence we have:

$$
\mathbf x'_{[\times]} \mathbf t = 
\begin{bmatrix}
0 & -x_3' & x_2' \\
x_3' & 0 & -x_1' \\
-x_2' & x_1' & 0 
\end{bmatrix} \mathbf t = -\frac{1}{x_4} \mathbf x' \times R \mathbf x_{1-3}
$$

$\mathbf x'_{[\times]}$ has rank 2 (it cannot be full rank since $\mathbf x'_{[\times]} \mathbf x' = \mathbf x' \times \mathbf x' = \mathbf 0$), at least two such constraints are needed in order to specify $\mathbf t$.

If the projections of the vanishing points are taken as known points, as they are the images of $\begin{pmatrix}1 & 0 & 0 & 1\end{pmatrix}^\top$ and $\begin{pmatrix}0 & 1 & 0 & 1\end{pmatrix}^\top$ respectively, the right hand side vector become $-\mathbf x' \times R^{1\top}$ and $-\mathbf y' \times R^{2\top}$ where $R^{n\top}$ id the $n$-th column of $R$.

In [ ]:
def find_projection_matrix(K, vpx, vpy):
    def normalize(v):
        return v/np.linalg.norm(v)
    
    def crossmatrix(u):
        return np.array([[0,    -u[2],  u[1]],
                         [u[2],   0,   -u[0]],
                         [-u[1],  u[0],  0 ]])
    
    # vpx_impl is the projection of vpx on the image plane, without Z coordinate
    vpx = np.array(vpx)
    vpy = np.array(vpy)
    K = np.array(K)
    
    Kinv = np.linalg.inv(K)
    vpx_impl = np.dot(Kinv, vpx)
    vpy_impl = np.dot(Kinv, vpy)
    vpx_impl = vpx_impl/vpx_impl[2]
    vpy_impl = vpy_impl/vpy_impl[2]
    print(vpx_impl, vpy_impl)
    
    # (lowercase) k is the Z coordinate of the plane, u and v are the full vectors
    k = np.sqrt(-vpx_impl[0]*vpy_impl[0] - vpx_impl[1]*vpy_impl[1])
    u = [vpx_impl[0], vpx_impl[1], k]
    v = [vpy_impl[0], vpy_impl[1], k]
    R = np.column_stack((normalize(u), normalize(v), normalize(np.cross(u, v))))
    
    bx = -np.cross(vpx_impl, R[:,0].T)
    by = -np.cross(vpy_impl, R[:,1].T)
    b = np.concatenate((bx, by))
    A = np.row_stack((crossmatrix(vpx_impl), crossmatrix(vpy_impl)))
    
    def minors(n): 
        return ([i, j, k] for i in range(n)
                          for j in range(i+1, n)
                          for k in range(j+1, n))
    
    # We select a triple of linearly independent rows
    ind_minor = next(m for m in minors(6) if np.linalg.det(A[m]) != 0)
    A = A[ind_minor]
    b = b[ind_minor]
    t = np.linalg.solve(A, b)
    
    P = np.dot(K, np.column_stack((R, t)))
    print(np.linalg.det(A))
    return P

In [ ]:
K = [[1159.10049262109, 8.81182868511778,  656.356931046326],
     [0,                1164.35813268503,  499.093066111278],
     [0,                0,                 1               ]]
vpx = [ 4.03377304e+03, -1.03233832e+03,  1.00000000e+00]
vpy = [-7.78844385e+02, -1.86022864e+03,  1.00000000e+00]

In [ ]:
find_projection_matrix(K, vpx, vpy)

## Deriving P with method 4.1 from the paper

$\lambda_h$ can be derived imposing the area of the table surface as a constraint:

$$
AB \cdot AC = A \\
A = |\mathbf p_B - \mathbf p_A|\cdot|\mathbf p_C - \mathbf p_A| \\
A = |\lambda_h \mathbf c_B - \lambda_h \mathbf c_A|\cdot|\lambda_h \mathbf c_C - \lambda_h \mathbf c_A| \\
\lambda_h = \sqrt \frac{A}{|\mathbf c_B - \mathbf c_A|\cdot|\mathbf c_C - \mathbf c_A|}
$$

$\mathbf c_A$ is the vector obtaining in equation (15) of the paper by decomposing $\mathbf t_{wc}$ as in equation (13):

$$
\mathbf p_A =
\frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \mathbf t_{wc}}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a =
\frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \lambda_h \hat{\mathbf q}_h}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a =
\lambda_h \frac{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_h}{\hat{\mathbf q}_n \times \hat{\mathbf q}_m \cdot \hat{\mathbf q}_a} \hat{\mathbf q}_a
=: \lambda_h \mathbf c_A
$$

In [ ]:
# All arguments are 3D vectors
def corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qa_norm):
    return np.dot(np.cross(qn_norm, qm_norm), qh_norm)/np.dot(np.cross(qn_norm, qm_norm), qa_norm) * qa_norm

# Normalized vector in camera coordinates corresponding to point in pixel coordinates
def qnorm(u_px, v_px, K, k):
    uv = np.dot(np.linalg.inv(K), [u_px, v_px, 1])
    q = uv/uv[2]
    q[2] = k
    return q/np.linalg.norm(q)

In [ ]:
# Vertex A of the table must be consecutive to both vertices B and C;
# H is the center of the table.
# vpx, vpy, uv* must be given in homogeneous pixel coordinates.
def find_projection_matrix2(K, k, vpx, vpy, uva, uvb, uvc, uvh, table_area):
    def normalize(v):
        return v/np.linalg.norm(v)
    
    def hom2eucl(v):
        assert(v[-1] != 0), f"Point at infinity {v} does not have an Euclidean correspondent!"
        return v[:-1]/v[-1]
    
    # vpx_impl is the projection of vpx on the image plane, without Z coordinate
    vpx = np.array(vpx)
    vpy = np.array(vpy)
    K = np.array(K)
    
    # (lowercase) k is the Z coordinate of the plane, u and v are the full vectors
    qm_norm = qnorm(*hom2eucl(vpx), K, k)
    qn_norm = qnorm(*hom2eucl(vpy), K, k)
    qz_norm = np.cross(qm_norm, qn_norm)
    R = np.column_stack(qm_norm, qn_norm, qz_norm)
    
    qh_norm = qnorm(*hom2eucl(uvh), K, k)
    qa_norm = qnorm(*hom2eucl(uva), K, k)
    qb_norm = qnorm(*hom2eucl(uvb), K, k)
    qc_norm = qnorm(*hom2eucl(uvc), K, k)
    
    ca = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qa_norm)
    cb = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qb_norm)
    cc = corner_lambda_h_coeff(qm_norm, qn_norm, qh_norm, qc_norm)
    
    lambda_h = np.sqrt(table_area/(np.linalg.norm(cb - ca)*np.linalg.norm(cc - ca)))
    t = lambda_h * qh_norm
    
    P = np.dot(K, np.column_stack((R, t)))
    print(np.linalg.det(A))
    return P